# Complex System Simulation: Corals and Flows
#### _Group 12: Esther Bakels (12362980), Loes Bijman (15211312), Aleksandar Jacimovic (), Boyan Mihaylov (15067602)_

In [ ]:
import matplotlib.pyplot as plt

from dla_model import *